In [3]:
from source.mb_base import MusicBert

BERT = MusicBert(dim_sound = 20, name = "music_bert_classification")

In [19]:
from sklearn import svm

import numpy as np
from source.subtasks_datasets import *
from source.utils import *

with open("data/emoMusic/train.npy", "rb")as f:
    track_list = np.load(f, allow_pickle=True)

X = np.array([np.mean(item['mfcc'], axis=0) for item in track_list ])
Y = np.array([item['emo_values'][::2] for item in track_list ])
    
print(X.shape, Y.shape)

clf = []
for i in range(2):
    clf.append(svm.SVR())

    clf[i].fit(X, Y[:,i])

(619, 20) (619, 2)


In [22]:
from sklearn.metrics import r2_score

with open("data/emoMusic/test.npy", "rb")as f:
    track_list = np.load(f, allow_pickle=True)

X = np.array([np.mean(item['mfcc'], axis=0) for item in track_list ])
Y = np.array([item['emo_values'][::2] for item in track_list ])

for i in range(2):
    print(r2_score(Y[:,i], clf[i].predict(X), multioutput='raw_values'))

[0.48314546]
[0.01180382]


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from torch.nn.utils.rnn import pad_sequence

from source.subtasks_datasets import *
from source.utils import just_load
from tqdm.notebook import tqdm


train_dataset = EmoMusicDataset(filename="train.npy")
test_dataset = EmoMusicDataset(filename="test.npy")

def svm_sequence_regression(model, train_dataset, test_dataset=None, eval_fn=None):
    
    ## eval_fn: function that takes (true_labels, predictions) and returns a score 
    
    if test_dataset is None:
        train_dataset, test_dataset = train_test_split(train_dataset, test_size=0.2)
    
    def preprocess(dataset):
        dataloader = just_load(dataset, shuffle=False)
        
        features = []
        for batch in tqdm(dataloader):
            x = batch['song_features'].permute(1,0,2).to(BERT.get_device())
            out = BERT(x)[0] # Takes just the classification token for the sequence
            
            features.append(out.detach())
        
        
        return np.array(features)
    
    def get_labels(dataset):
        Y = np.array([item['target'] for item in dataset])
        return Y
    
    train_X, test_X = preprocess(train_dataset), preprocess(test_dataset)
    train_Y, test_Y = get_labels(train_dataset), get_labels(test_dataset)
    
    # Assume that the out_shape is of the form [Batch, Out_Shape] or [Batch]
    out_shape = Y.shape
    if len(out_shape) == 1:
        n_targets = 1
    else:
        n_targets = out_shape[-1]
        
    model = MultiOutputRegressor(SVR())
    model.fit(train_X, train_Y)
    
    pred_Y = model.predict(test_X)
    
    if eval_fn is None:
        return r2_score(test_Y, pred_Y, multioutput='raw_values')
    else:
        return eval_fn(test_Y, pred_Y)
    
svm_sequence_regression(BERT, train_dataset, test_dataset=test_dataset)

NameError: name 'get' is not defined